In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
train_data=pd.read_csv(r"C:\Users\harim\Downloads\toxic\train\train.csv")
test_data=pd.read_csv(r"C:\Users\harim\Downloads\toxic\test\test.csv")

In [2]:
import re
abb = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}
#defining a function to clean the data
def clean_text(text):
    
    text = text.lower()
    text = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '', text) # clean url
    text = re.sub(r'#(\w+)', '', text)   # clean hashes
    text = re.sub(r'@(\w+)', '', text)   # clean @
    text = re.sub(r'<[^>]+>', '', text)  # clean tags
    text = re.sub(r'\d+', '', text)      # clean digits
    text = re.sub(r'[,!@\'\"?\.$%_&#*+-:;]', '', text)   # clean punctuation
    text = [abb[word] if word in abb else word for word in text.split()]  #
    
    return text
#applying the cleantext function to bothe train and test data
train_data['comment_text'] = train_data['comment_text'].apply(clean_text)
test_data['comment_text'] = test_data['comment_text'].apply(clean_text)

In [3]:
sentences = train_data["comment_text"].fillna("DUMMY_VALUE").values
possible_labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
targets = train_data[possible_labels].values

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tk = Tokenizer(num_words = 100000, oov_token='<oov>')
tk.fit_on_texts(train_data)
training_sequences = tk.texts_to_sequences(sentences)
training_padded = pad_sequences(training_sequences, maxlen = 100,
                                padding = 'pre',
                                truncating='pre')

In [ ]:
from sklearn.model_selection import train_test_split
X_train =training_padded
y_train=targets
X_train, X_test, y_train, y_test = train_test_split(training_padded,targets, test_size=0.25, random_state=42)


In [6]:
from keras.layers import Dense,LSTM,Bidirectional,Dropout,Embedding,GlobalMaxPooling1D,Conv1D
from keras.models import Sequential

vocab_size=len(tk.word_index)+1

In [7]:
model=Sequential()
model.add(Embedding(vocab_size,100))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Conv1D(128,3,padding="valid"))
model.add(GlobalMaxPooling1D())

model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(256,activation="relu"))
model.add(Dense(6,activation="sigmoid"))

In [8]:
model.compile(loss="binary_crossentropy",optimizer=keras.optimizers.Adam(lr=3e-4),metrics=['accuracy'])

In [9]:
model.fit(X_train,y_train,batch_size=64,epochs=2,shuffle=True)

Epoch 1/2
1746/1746 [==============================] - 833s 472ms/step - loss: 0.1703 - accuracy: 0.8554
Epoch 2/2
1746/1746 [==============================] - 838s 480ms/step - loss: 0.1399 - accuracy: 0.9940


In [10]:
model.evaluate(X_train,y_train)

3491/3491 [==============================] - 261s 74ms/step - loss: 0.1411 - accuracy: 0.9942


[0.1410616785287857, 0.9942166209220886]

In [12]:
from sklearn.metrics import accuracy_score,confusion_matrix
pred=model.predict(X_test)
pred

array([[0.11175269, 0.00714377, 0.05747572, 0.00232637, 0.05602905,
        0.00863495],
       [0.1247071 , 0.0087392 , 0.06570429, 0.00277424, 0.06453511,
        0.01020432],
       [0.19008335, 0.02159321, 0.11836749, 0.00688446, 0.11042944,
        0.02163625],
       ...,
       [0.19651943, 0.02343571, 0.12411308, 0.00750005, 0.11495152,
        0.02317205],
       [0.19644299, 0.02343783, 0.12401801, 0.0075126 , 0.11491674,
        0.02320275],
       [0.19232452, 0.02223459, 0.12035707, 0.00710788, 0.11209419,
        0.02219126]], dtype=float32)